# The Sundial Problem from a New Angle

This notebook accompanies my [sundial paper](https://russellgoyder.github.io/sundial-latex/) by the same name. I originally used [Maple](https://www.maplesoft.com/) for the calculations, but decided to reproduce them here to make the analysis more accessible, and as away to learn [SymPy](https://www.sympy.org/en/index.html) and [GAlgebra](https://github.com/pygae/galgebra).

I haven't reproduced the orbit analysis, equation of time or plots of the shadow tip yet, but plan to add them in the future. For now, however, all the main results are reproduced below.

In [1]:
import sympy as sp
from sympy import sin, cos, tan
from sympy.abc import *
from galgebra import metric, mv
from galgebra.ga import Ga
from galgebra.printer import latex
from IPython.display import Math

# tell sympy to use galgebra printing
sp.init_printing(
    latex_printer=latex,
    use_latex='mathjax',
    omit_function_args=True,
    omit_partial_derivative_fraction=True,
    )

## Setup and Definitions

### Fixed Stars Frame

First, we define the geometric algebra of 3-space and some basis blades from the frame of the "fixed stars"

![Earth's orientation and orbit](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/MainArena.png?token=GHSAT0AAAAAAB73Q3JM6JGFDMRHPPAJKRQ2ZAWFO7Q "Earth's orientation and orbit.").

In [2]:
coords = sp.symbols('1 2 3', real=True)
G3 = Ga('e', g=[1,1,1], coords=coords)

(e1, e2, e3) = G3.mv()
I = e1^e2^e3

### Earth Frame

Now, let the tilt of the earth's plane (axis) of rotation be $\alpha$ and measure the earth's rotation by $\psi$. Then we can define the Earth frame as follows.

In [3]:
R_alpha = cos(alpha/2) - (e1^e3) * sin(alpha/2)

def rotate(mv, rotor):
    return ( rotor * mv * rotor.rev() )

e1_prime = rotate(e1, R_alpha).trigsimp()
R_psi = (cos(psi/2) - (e1_prime^e2 * sin(psi/2))).trigsimp()

f1 = rotate(e1_prime, R_psi).trigsimp().trigsimp()
f2 = rotate(e2, R_psi).trigsimp().trigsimp()
f3 = rotate(e3, R_alpha).trigsimp().trigsimp()

def print_3frame(frame, symbol):
    return Math(fr'''
            \begin{{align}}
            {symbol}_1 &= {latex(frame[0])} \nonumber \\
            {symbol}_2 &= {latex(frame[1])} \nonumber \\
            {symbol}_3 &= {latex(frame[2])} \nonumber
            \end{{align}}
            ''')

print_3frame((f1,f2,f3), "f")

<IPython.core.display.Math object>

Check that this is an orthonormal frame

In [4]:
result = (e1 ^ e2 ^ e3) - (f1 * f2 * f3)
assert result.obj.equals(0)

The equatorial plane should only depend on the tilt of the Earth's axis of spin $\alpha$, not the angle by which it has rotated relative to the fixed stars $\psi$.

In [5]:
f1^f2

cos(alpha)*e_1^e_2 - sin(alpha)*e_2^e_3

### Surface Frame

Define an orthonormal frame embedded in the Earth's surface, with $n_1$ pointing South, $n_2$ pointing East and $n_3$ pointing up.

![](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/SurfaceFrame.png?token=GHSAT0AAAAAAB73Q3JNXNOAJWYLCUINTVLUZAWF6JQ "Frame embedded in Earth's surface.").

<!-- ![](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/SurfaceFrame.svg?token=GHSAT0AAAAAAB73Q3JNXNOAJWYLCUINTVLUZAWF6JQ "Frame embedded in Earth's surface."). -->

In [6]:
R_theta = cos(theta/2) - ( (f3^f1) * sin(theta/2) )

n1 = rotate(f1, R_theta).trigsimp().trigsimp()
n2 = f2

# n3 needs a little love
raw_n3 = rotate(f3, R_theta).obj.trigsimp()
sympy_n3 = sp.expand(sp.expand_trig(raw_n3)) # galgebra's Mv doesn't have expand_trig as a method
n3 = mv.Mv(sympy_n3, ga=G3)

print_3frame((n1,n2,n3), "n")

<IPython.core.display.Math object>

Check that this basis is orthnormal. Geometric product of the $\{n_i\}$ should give the pseudoscalar $I = e_1 \wedge e_2 \wedge e_3$

In [7]:
(n1*n2*n3).trigsimp()


e_1^e_2^e_3

### Orbit Rotor and Meridian Plane

Earth orbit rotor $R_\sigma$, and vector parallel to rays of sunshine, $s$.

In [8]:
R_sigma = cos(sigma/2) - (e1^e2)*sin(sigma/2)
s = rotate(e1, R_sigma).trigsimp()

def print_eq(lhs : str, rhs : sp.Symbol):
    return Math(fr'''
        \begin{{equation}}
            {lhs} = {latex(rhs)} \nonumber
        \end{{equation}}
        ''')

s.Fmt(1, "s")

s = cos(sigma)*e_1 + sin(sigma)*e_2

The meridian plane, $M$.

In [9]:
M = (n1^n3).trigsimp()
M.Fmt(1, "M")

M = sin(alpha)*sin(psi)*e_1^e_2 + cos(psi)*e_1^e_3 + sin(psi)*cos(alpha)*e_2^e_3

The noon line is the intersection of the sunshine vector $s$ and the meridian plane $M$, which occurs where $s \wedge M$ vanishes.

In [10]:
(s^M).trigsimp()

(sin(psi)*cos(alpha)*cos(sigma) - sin(sigma)*cos(psi))*e_1^e_2^e_3

In [11]:
coeff = _.trigsimp().get_coefs(3)[0]
soln = sp.solve(coeff.subs(sin(psi), tan(psi)*cos(psi)), tan(psi))[0]
print_eq( r"\tan(\psi)", soln )

<IPython.core.display.Math object>

### Dial face and gnomon

Define an orthnormal frame $u_1, u_2, u_3$ as the unevaluated version of $n_1, n_2, n_3$.

![](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/DialFrame.png?token=GHSAT0AAAAAAB73Q3JNJN46TIEHP3QCWWGYZAWGADA "Frame embedded in the sundial's face.").


In [12]:
G3u = Ga('u', g=[1,1,1], coords=coords)

(u1, u2, u3) = G3u.mv()
U = u1^u2^u3
U

u_1^u_2^u_3

Dial face expressed relative to $u$ basis: $G_u$.

In [13]:
R_i = cos(i/2) - (u1^u3)*sin(i/2)
R_d = cos(d/2) - (u1^u2)*sin(d/2)

Gu = rotate(rotate(u1^u2, R_i), R_d).trigsimp()
Gu.Fmt(1, "G_u")

G_u = cos(i)*u_1^u_2 + sin(d)*sin(i)*u_1^u_3 - sin(i)*cos(d)*u_2^u_3

Frame embedded in dial face.

In [14]:
m1 = rotate(rotate(u1, R_i), R_d).trigsimp()
m2 = rotate(rotate(u2, R_i), R_d)
m3 = rotate(rotate(u3, R_i), R_d).trigsimp()

print_3frame((m1,m2,m3), "m")

<IPython.core.display.Math object>

The gnomon expressed relative to the $u$ frame, $g_u$.

![](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/Gnomon.png?token=GHSAT0AAAAAAB73Q3JNHJMDP6T55SWPTQFGZAWGBNA "The gnomon.").


In [15]:
R_iota = cos(iota/2) - (u1^u3)*sin(iota/2)
R_delta = cos(delta/2) - (u1^u2)*sin(delta/2)

gu = rotate(rotate(u3, R_iota), R_delta).trigsimp()

# extra manipulation to display exactly as in paper
print_eq("g_u", sp.collect(sp.trigsimp(gu.obj), -sin(iota)))

<IPython.core.display.Math object>

Projected onto the fixed-stars basis, the gnomon is

In [16]:
g = sum([ c*ni for c, ni in zip(gu.get_coefs(1),[n1, n2, n3])])
g = g.trigsimp()
g.Fmt(3, "g")

g =  (-sin(alpha)*sin(iota)*sin(theta)*cos(delta) - sin(alpha)*cos(iota)*cos(theta) + sin(delta)*sin(iota)*sin(psi)*cos(alpha) - sin(iota)*cos(alpha)*cos(delta)*cos(psi)*cos(theta) + sin(theta)*cos(alpha)*cos(iota)*cos(psi))*e_1
 + (-sin(delta)*sin(iota)*cos(psi) - sin(iota)*sin(psi)*cos(delta)*cos(theta) + sin(psi)*sin(theta)*cos(iota))*e_2
 + (sin(alpha)*sin(delta)*sin(iota)*sin(psi) - sin(alpha)*sin(iota)*cos(delta)*cos(psi)*cos(theta) + sin(alpha)*sin(theta)*cos(iota)*cos(psi) + sin(iota)*sin(theta)*cos(alpha)*cos(delta) + cos(alpha)*cos(iota)*cos(theta))*e_3

The gnomon lies in the meridian plane when the following trivector vanishes:

In [17]:
M^g

sin(delta)*sin(iota)*e_1^e_2^e_3

Will also want $s$ on the $n$ frame

In [18]:
coeffs = [sp.trigsimp((s|c).obj) for c in [n1, n2, n3]]
su = sum([coeff*vec for coeff, vec in zip(coeffs, [u1, u2, u3])])
su.Fmt(3, "s_u")


s_u =  (sin(alpha)*sin(theta)*cos(sigma) + sin(psi)*sin(sigma)*cos(theta) + cos(alpha)*cos(psi)*cos(sigma)*cos(theta))*u_1
 + (-sin(psi)*cos(alpha)*cos(sigma) + sin(sigma)*cos(psi))*u_2
 + (-sin(alpha)*cos(sigma)*cos(theta) + sin(psi)*sin(sigma)*sin(theta) + sin(theta)*cos(alpha)*cos(psi)*cos(sigma))*u_3

## The Calculation

Work in the case where its declination angle $\delta$ is zero.

In [19]:
gu_delta = gu
gu = gu.subs(delta, 0)
gu.Fmt(1, "g_u")

g_u = -sin(iota)*u_1 + cos(iota)*u_3

In [20]:
g_delta = g
g = g.subs(delta, 0)
g.Fmt(1, "g")

g = (-sin(alpha)*cos(iota - theta) - sin(iota - theta)*cos(alpha)*cos(psi))*e_1 - sin(psi)*sin(iota - theta)*e_2 + (-sin(alpha)*sin(iota - theta)*cos(psi) + cos(alpha)*cos(iota - theta))*e_3

`g.obj` trails behind by one trigsimp

In [21]:
g.obj

(-sin(iota)*sin(psi)*cos(theta) + sin(psi)*sin(theta)*cos(iota))*e_2 + (-sin(alpha)*sin(iota)*sin(theta) - sin(alpha)*cos(iota)*cos(theta) - sin(iota)*cos(alpha)*cos(psi)*cos(theta) + sin(theta)*cos(alpha)*cos(iota)*cos(psi))*e_1 + (-sin(alpha)*sin(iota)*cos(psi)*cos(theta) + sin(alpha)*sin(theta)*cos(iota)*cos(psi) + sin(iota)*sin(theta)*cos(alpha) + cos(alpha)*cos(iota)*cos(theta))*e_3

Hack: ensure a healthy `g.obj` by reforming $g$ from the result of applying `sympy.trigsimp` to each component individually

In [22]:
g = sum([sp.trigsimp(coef)*vec for coef, vec in zip(g.get_coefs(1), G3.mv())])
g

(-sin(alpha)*cos(iota - theta) - sin(iota - theta)*cos(alpha)*cos(psi))*e_1 - sin(psi)*sin(iota - theta)*e_2 + (-sin(alpha)*sin(iota - theta)*cos(psi) + cos(alpha)*cos(iota - theta))*e_3

In [23]:
# eyeball check:
g.obj

-sin(alpha)*sin(iota - theta)*cos(psi)*e_3 - sin(alpha)*cos(iota - theta)*e_1 - sin(psi)*sin(iota - theta)*e_2 - sin(iota - theta)*cos(alpha)*cos(psi)*e_1 + cos(alpha)*cos(iota - theta)*e_3

### The Shadow Plane

$S = s \wedge g$ is the plane containing the sunshine vector and the gnomon.

In [24]:
S = s^g
S.Fmt(3, "S")

S =  (sin(alpha)*sin(sigma)*cos(iota - theta) - sin(psi)*sin(iota - theta)*cos(sigma) + sin(sigma)*sin(iota - theta)*cos(alpha)*cos(psi))*e_1^e_2
 + (-sin(alpha)*sin(iota - theta)*cos(psi) + cos(alpha)*cos(iota - theta))*cos(sigma)*e_1^e_3
 + (-sin(alpha)*sin(iota - theta)*cos(psi) + cos(alpha)*cos(iota - theta))*sin(sigma)*e_2^e_3

Cosine of angle $\Xi$ between sun ray and gnomon:

In [25]:
cosXi = sp.trigsimp((s|g).obj)
print_eq(r"\cos(\Xi)", cosXi)

<IPython.core.display.Math object>

Check $S^2 = (s\wedge g)^2 = (s\cdot g)^2 - s^2 g^2$ where $s^2 = g^2 = 1$. 

In [26]:
sg_squared = (s|g)*(s|g)
(S|S) - ( sg_squared - 1 )

0

The magnitude of $S$ is given by $\sqrt{-S^2} = \sqrt{1 - (s\cdot g)^2} = \sqrt{1 - \cos^2(\Xi}) = \sin^2(\Xi)$:

In [27]:
sinXi = sp.sqrt( 1 - ((cos(alpha)*cos(sigma)*cos(psi) + sin(sigma)*sin(psi))*sin(iota-theta) + sin(alpha)*cos(sigma)*cos(iota-theta))**2 )
print_eq("\sin(\Xi)", sinXi)

<IPython.core.display.Math object>

In [28]:
# check
sinXi**2 - ( 1 - sg_squared )

0

### The Hour Angle

Define a generalized solar hour angle $\mu$ as the angle between $S$ and $M$, given by
$$\cos(\mu) = \frac{-S\cdot M}{\sqrt{-S^2}\sqrt{-M^2}} = \frac{-S\cdot M}{\sin(\Xi)}$$
given that $M^2=-1$

In [29]:
sinXi_cos_mu = sp.trigsimp(-(S|M).obj)

print_eq( r"\sin(\Xi) \cos(\mu)", sinXi_cos_mu )

<IPython.core.display.Math object>

From $\sin^2\mu + \cos^2\mu = 1$ (by hand), we have:

In [30]:
sinXi_sin_mu = cos(alpha)*sin(psi)*cos(sigma) - cos(psi)*sin(sigma)
print_eq("\sin(\Xi) \sin(\mu)", sinXi_sin_mu)

<IPython.core.display.Math object>

Check that $\sin^2(\Xi)\sin^2(\mu) + \sin^2(\Xi)\cos^2(\mu) = \sin^2(\Xi)$

In [31]:
sp.trigsimp( sp.expand(sinXi_sin_mu**2) + sp.expand(sinXi_cos_mu**2) - sp.expand(sinXi**2) )

0

The ratio gives $\tan(\mu)$ as in the paper:

In [32]:
mu_ratio = sinXi_sin_mu / sp.collect(sinXi_cos_mu, cos(iota-theta))
print_eq( r"\tan(\mu)", sp.simplify( mu_ratio.subs(sin(sigma), tan(sigma)*cos(sigma)) ) )

<IPython.core.display.Math object>

Now express $S$ in terms of $\mu$ on the $\{n\}$ basis. First remind myself what $S$ looks like:

In [33]:
S.Fmt(3, "S")

S =  (sin(alpha)*sin(sigma)*cos(iota - theta) - sin(psi)*sin(iota - theta)*cos(sigma) + sin(sigma)*sin(iota - theta)*cos(alpha)*cos(psi))*e_1^e_2
 + (-sin(alpha)*sin(iota - theta)*cos(psi) + cos(alpha)*cos(iota - theta))*cos(sigma)*e_1^e_3
 + (-sin(alpha)*sin(iota - theta)*cos(psi) + cos(alpha)*cos(iota - theta))*sin(sigma)*e_2^e_3

and save nice forms of the $n$ bivectors

In [34]:
n12 = mv.Mv(sp.trigsimp(sp.expand_trig((n1^n2).obj)), ga=G3)
n13 = mv.Mv(sp.trigsimp(sp.expand_trig((n1^n3).obj)), ga=G3)
n23 = mv.Mv(sp.trigsimp(sp.expand_trig((n2^n3).obj)), ga=G3)

So now can check each component of S on the $n$ basis

In [35]:
(S|n12) + (-sinXi_sin_mu*sin(iota)) == (S|n13) + sinXi_cos_mu == (S|n23) + (-sinXi_sin_mu*cos(iota)) == mv.Mv(0, ga=G3)

True

and confirm S in this form

In [36]:
S - (-sinXi_sin_mu*(sin(iota)*n12 + cos(iota)*n23) + sinXi_cos_mu*n13)

0

so can capture S in terms of the surface-frame bivectors

In [37]:
Xi = sp.Symbol("\Xi")
Su = sin(Xi)*( -sin(mu) * (sin(iota)*(u1^u2) + cos(iota)*(u2^u3)) + cos(mu)*(u1^u3) )
Su.Fmt(1, "S_u")

S_u = -sin(\Xi)*sin(iota)*sin(mu)*u_1^u_2 + sin(\Xi)*cos(mu)*u_1^u_3 - sin(\Xi)*sin(mu)*cos(iota)*u_2^u_3

Check that $S_u$ and $S$ are really the same thing

In [38]:
def e_frame(bvec):
    """
    Take a bivector in the GA defined by the surface frame n1, n2, n3 (written as u1, u2, u3)
    and express it in the GA defined by the frame of the fixed stars, e1, e2, e3
    """
    coeffs = bvec.get_coefs(2)
    return coeffs[0]*n12 + coeffs[1]*n13 + coeffs[2]*n23

e_frame(Su).subs(sin(mu), sinXi_sin_mu/sin(Xi)).subs(cos(mu), sinXi_cos_mu/sin(Xi)) - S


0

### The Shadow Angle

Form a vector parallel to the shadow, as the intersection between $S$ (the plane containing the sun ray and the gnomon) and $G$ (the dial face)

In [39]:
u = ((u1^u2^u3) * Gu * Su).get_grade(1)
u.Fmt(3, "u")

u =  (sin(d)*sin(i)*sin(iota)*sin(mu) + cos(i)*cos(mu))*sin(\Xi)*u_1
 + (-sin(i)*sin(iota)*cos(d) - cos(i)*cos(iota))*sin(\Xi)*sin(mu)*u_2
 + (-sin(d)*sin(mu)*cos(iota) + cos(d)*cos(mu))*sin(\Xi)*sin(i)*u_3

Want to normalize $u$ to get $\hat{u} \equiv \hat{w}$, a unit vector parallel to the shadow $w$. Can square and add the above components, but there is a route to a simpler expression as follows.

The cosine of the angle between the "shadow plane" $S$ (containing the sun ray $s$ and the gnomon $g$) and dial face $G$ is given by
$$\cos(\Psi) = \frac{S\cdot G}{\sqrt{-S^2}\sqrt{-G^2}}$$
Given that $S^2 = -\sin^2(\Xi)$ and $G^2 = -1$,
$$ \cos(\Psi) = \frac{S\cdot G}{\sin(\Xi)}$$

In [40]:
cosPsi = sp.trigsimp(((Su|Gu)/sin(Xi)).obj)
print_eq(r"\cos(\Psi)", cosPsi)

<IPython.core.display.Math object>

In [41]:
# check (force sympy to give up the positive square root in each factor in the denominator)
sp.trigsimp((Su|Gu).obj) \
/ sp.powdenest(sp.sqrt(sp.trigsimp((-Su|Su).obj)), force=True) \
/ sp.powdenest(sp.sqrt(sp.trigsimp((-Gu|Gu).obj)), force=True) \
- ((Su|Gu)/sin(Xi))

0

Now, the length of $u$ is related to $\Psi$ as follows.
$$u^2 = (I\; G \times S)^2 = (\frac{I}{2}(GS-SG))^2 = -\frac{1}{4}(GSGS+SGSG-2S^2G^2)$$
But
$$(S\cdot G)^2 = \frac{1}{4}(SG+GS)^2 = \frac{1}{4}(SGSG+GSGS+2S^2G^2)$$
So,
$$u^2 = S^2G^2 - (S\cdot G)^2 = \sin^2(\Xi) - (S\cdot G)^2 = \sin^2(\Xi)\sin^2(\Psi)$$

In [42]:
# check
(u|u) - (sin(Xi)**2 - (Su|Gu)**2)

0

We can now form $\hat{w}$ by dividing $u$ by its length

In [43]:
Psi = sp.Symbol("\Psi")
what = u/sin(Xi)/sin(Psi)
what.Fmt(3, r"\hat{w}")

\hat{w} =  (sin(d)*sin(i)*sin(iota)*sin(mu) + cos(i)*cos(mu))*u_1/sin(\Psi)
 - (sin(i)*sin(iota)*cos(d) + cos(i)*cos(iota))*sin(mu)*u_2/sin(\Psi)
 + (-sin(d)*sin(mu)*cos(iota) + cos(d)*cos(mu))*sin(i)*u_3/sin(\Psi)

Check that
$$\hat{w}^2 = \left(\frac{u}{\sin(\Xi)\sin(\Psi)}\right)^2 = 1$$
by showing that
$$\frac{u^2}{\sin^2(\Xi)} - \sin^2\Psi = 0$$

In [46]:
sinPsiSquared = 1 - sp.expand(cosPsi**2)
u_over_sinXi_squared = sp.trigsimp(((u|u)/sin(Xi)**2).obj)
sp.trigsimp(u_over_sinXi_squared - sinPsiSquared)

0

Get the angular coordinate of the shadow tip relative to noon, call it $\zeta$. At noon we have $\mu = 0$

In [48]:
sinPsi_noon = sp.sqrt(1-cosPsi**2).subs(mu,0)
sinPsi_noon

sqrt(-sin(d)**2*sin(i)**2 + 1)

In [49]:
noon = what.subs(sin(Psi),sinPsi_noon).subs(mu,0)
noon

cos(i)*u_1/sqrt(-sin(d)**2*sin(i)**2 + 1) + sin(i)*cos(d)*u_3/sqrt(-sin(d)**2*sin(i)**2 + 1)

So $\cos(\zeta) = \hat{w}(\mu) \cdot \hat{w}(0)$

In [50]:
cos_zeta = sp.collect(sp.collect(sp.trigsimp((what|noon).obj),sin(mu)*sin(i)*sin(d)),cos(mu))
print_eq( r"\cos(\zeta)", cos_zeta )

<IPython.core.display.Math object>

And easiest route to $\sin(\zeta)$ is to use the fact that the shadow lives in the plane of the dial face, so $\hat{w}(\mu) \wedge \hat{w}(0) = \sin(\zeta) G$

In [51]:
sin_zeta = -(what ^ noon) | Gu
print_eq(r"\sin(\zeta)", sin_zeta)

<IPython.core.display.Math object>

In [52]:
tan_zeta = (sin_zeta/cos_zeta).subs(sin(mu), tan(mu)*cos(mu))
print_eq(r"\tan(\zeta)", tan_zeta)

<IPython.core.display.Math object>

Check that $\sin^2(\zeta) + \cos^2(\zeta) = 1$

In [63]:
sp.simplify(sp.expand( sp.trigsimp(sin_zeta.obj)**2 + cos_zeta**2 ))

(-sin(d)**2*sin(i)**2*sin(iota)**2*sin(mu)**2 + 2*sin(d)**2*sin(i)**2*sin(mu)**2 - sin(d)**2*sin(i)**2 - 2*sin(d)*sin(i)**2*sin(mu)*cos(d)*cos(iota)*cos(mu) + 2*sin(d)*sin(i)*sin(iota)*sin(mu)*cos(i)*cos(mu) + 2*sin(i)**2*sin(iota)**2*sin(mu)**2 - sin(i)**2*sin(mu)**2 + 2*sin(i)*sin(iota)*sin(mu)**2*cos(d)*cos(i)*cos(iota) - sin(iota)**2*sin(mu)**2 + 1)/sin(\Psi)**2

This is 1 if the numerator is equal to the denominator, and we have an explicit expression for $\cos(\Psi) = \sqrt{1 - \sin^2(\Psi)}$. So should get zero from the following:

In [64]:
sp.trigsimp(sp.numer(_) - (1-cosPsi**2))

0

### The Shadow Length

We have the angle of the shadow vector $w$. Now find the length. Form $w(p) = g + p*s$ and enforce $w(p) \wedge G = 0$ (used $\lambda$ instead of $p$ in the paper but can't here because it's a keyword in Python).

In [65]:
G = e_frame(Gu)
((g + p*s)^G) * I

p*sin(alpha)*sin(i)*sin(theta)*cos(d)*cos(sigma) + p*sin(alpha)*cos(i)*cos(sigma)*cos(theta) - p*sin(d)*sin(i)*sin(psi)*cos(alpha)*cos(sigma) + p*sin(d)*sin(i)*sin(sigma)*cos(psi) + p*sin(i)*sin(psi)*sin(sigma)*cos(d)*cos(theta) + p*sin(i)*cos(alpha)*cos(d)*cos(psi)*cos(sigma)*cos(theta) - p*sin(psi)*sin(sigma)*sin(theta)*cos(i) - p*sin(theta)*cos(alpha)*cos(i)*cos(psi)*cos(sigma) - sin(i)*sin(iota)*cos(d) - cos(i)*cos(iota)

In [66]:
p_soln = sp.solve(sp.trigsimp(_.obj), p)[0]
p_soln

(sin(i)*sin(iota)*cos(d) + cos(i)*cos(iota))/(sin(alpha)*sin(i)*sin(theta)*cos(d)*cos(sigma) + sin(alpha)*cos(i)*cos(sigma)*cos(theta) - sin(d)*sin(i)*sin(psi)*cos(alpha)*cos(sigma) + sin(d)*sin(i)*sin(sigma)*cos(psi) + sin(i)*sin(psi)*sin(sigma)*cos(d)*cos(theta) + sin(i)*cos(alpha)*cos(d)*cos(psi)*cos(sigma)*cos(theta) - sin(psi)*sin(sigma)*sin(theta)*cos(i) - sin(theta)*cos(alpha)*cos(i)*cos(psi)*cos(sigma))

We can write this is in terms of the hour angle $\mu$ evaluated under the condition $\iota = \theta$ and denoted $\mu_s$. This is true when the gnomon is parallel to the earth's axis, and termed a style, hence the subscript $s$.

In [67]:
mu_s = sp.Symbol(r"\mu_s")
D_soln = sin(Xi) * ((sin(i)*cos(d)*cos(theta)-cos(i)*sin(theta)) * cos(mu_s) - sin(i)*sin(d)*sin(mu_s)) + sin(alpha)*cos(sigma)*( sin(i)*sin(theta)*cos(d) + cos(i)*cos(theta) )
D_soln

((sin(i)*cos(d)*cos(theta) - sin(theta)*cos(i))*cos(\mu_s) - sin(\mu_s)*sin(d)*sin(i))*sin(\Xi) + (sin(i)*sin(theta)*cos(d) + cos(i)*cos(theta))*sin(alpha)*cos(sigma)

In [68]:
# check
sp.simplify(sp.denom(p_soln) - D_soln.subs(mu_s,mu).subs(sin(mu),sinXi_sin_mu/sin(Xi)).subs(cos(mu),sinXi_cos_mu/sin(Xi)).subs(iota,theta))

0

So given $p$, we have $w = g + ps$. However, it is a little cumbersome to work with and we can get $w$ in terms of $g$-$ps$-$w$ triangle directly by projecting $w$ onto $s$ and $g$ to give two equations we can solve for $p$ and $L$, the length of $w$. We have
$$s\cdot w = L\cos(\Xi-\beta) = s\cdot g + p\, s\cdot s = \cos(\Xi) + p$$
and
$$g\cdot w = L\cos(\beta) = g\cdot g + p g\cdot s = 1 + p\, \cos(\Xi)$$
This yields two simultaneous equations for $p$ and $L$, with solutions
$$p = \frac{\sin(\beta)}{\sin(\Xi-\beta)}$$
and
$$L = \frac{\sin(\Xi)}{\sin(\Xi-\beta)}$$
First define $w$ on the $n$ basis

In [69]:
wu = gu + p*su
wu.Fmt(3, "w")

w =  (p*sin(alpha)*sin(theta)*cos(sigma) + p*sin(psi)*sin(sigma)*cos(theta) + p*cos(alpha)*cos(psi)*cos(sigma)*cos(theta) - sin(iota))*u_1
 + p*(-sin(psi)*cos(alpha)*cos(sigma) + sin(sigma)*cos(psi))*u_2
 + (-p*sin(alpha)*cos(sigma)*cos(theta) + p*sin(psi)*sin(sigma)*sin(theta) + p*sin(theta)*cos(alpha)*cos(psi)*cos(sigma) + cos(iota))*u_3

Then check that $p + \cos(\Xi)$ is equal to the projection of $w$ onto $s$

In [70]:
(su|wu) - (p + cosXi)

0

Now check that $1 + p\cos(\Xi)$ is equal to the projection of $w$ onto $g$

In [71]:
(gu|wu) - (1 + p*cosXi)

0

Given that $\sin(\beta)$ appears in the solution for $p$, it will be useful to have its explicit form, which is

In [72]:
sin_beta = (sin(i)*sin(iota)*cos(d) + cos(i)*cos(iota))/sin(Psi)
print_eq(r"\sin(\beta)", sin_beta)

<IPython.core.display.Math object>

Confirm this by checking that $\sin^2(\beta) + \cos^2(\beta) - 1 = 0$

In [73]:
cos_beta = sp.collect(sp.trigsimp((gu|what).obj), cos(mu))
print_eq(r"\cos(\beta)", cos_beta)

<IPython.core.display.Math object>

In [74]:
sp.trigsimp(sp.expand(sp.numer(sin_beta)**2 + sp.numer(cos_beta)**2) - (1-cosPsi**2))

0

Give $\tan(\beta)$ as in the paper

In [75]:
print_eq(r"\tan(\beta)", sin_beta/cos_beta)

<IPython.core.display.Math object>

We can now use the form of $p$ found by solving $w\wedge G = 0$ to obtain $\sin(\Xi)$. First note that $N$, the numerator of $p$, is related to $\beta$ via
$$\sin(\beta) = \frac{N}{\sin(\Psi)}$$
Check

In [76]:
sp.numer(p_soln)/sin(Psi) - sin_beta

0

Given our solution for $p$, we can write
$$p = \frac{\sin(\beta)}{\sin(\Xi-\beta)} = \frac{N}{D}$$
Eliminating $N$ using the above relationship with $\beta$ and solving for $D$ gives
$$D = \sin(\Psi)\sin(\Xi-\beta)$$
Note that using this to eliminate $\sin(\Xi-\beta)$ in our solution for $L$ gives
$$L = \frac{\sin(\Xi)\sin(\Psi)}{D}$$
Expanding the double angle in $D$ and solving for $\sin(\Xi)$ gives
$$\sin(\Xi) = \frac{D + N\cos(\Xi)}{B}$$
where
$$\cos(\beta) = \frac{B}{\sin(\Psi)}$$
Thus

In [82]:
tmp = (sp.denom(p_soln) + sin(beta)*sin(Psi)*cos(Xi))/(cos(beta)*sin(Psi))
sinXi = tmp.subs(sin(beta),sin_beta).subs(cos(beta),cos_beta)
print_eq( r"\sin(\Xi)", sinXi)

<IPython.core.display.Math object>

### The Shadow Coordinates

Not very exciting. Let's now get the cartesian coordinates of the shadow tip in the dial face frame.
$$w = L \hat{w}$$
and so can work with $\hat{w}$ and scale up by after.

In [79]:
(what|m1).Fmt(1, r"\frac{x}{L}")

\frac{x}{L} = (-sin(d)*sin(mu)*cos(iota) + cos(d)*cos(mu))/sin(\Psi)

In [80]:
mv.Mv(sp.trigsimp(sp.expand_trig((what|m2).obj)), ga=G3u).Fmt(1, r"\frac{y}{L}")

\frac{y}{L} = -(sin(d)*cos(i)*cos(mu) + sin(i)*sin(iota)*sin(mu) + sin(mu)*cos(d)*cos(i)*cos(iota))/sin(\Psi)

In [81]:
(what|m3).Fmt(1, r"\frac{z}{L}")

\frac{z}{L} = 0